# Few-shot learning from thaigpt-next-125m

In [1]:
!pip install transformers

     |████████████████████████████████| 2.5MB 6.7MB/s 
     |████████████████████████████████| 3.3MB 39.2MB/s 
     |████████████████████████████████| 901kB 49.8MB/s 


Thank you code from https://medium.com/geekculture/fine-tune-eleutherai-gpt-neo-to-generate-netflix-movie-descriptions-in-only-47-lines-of-code-40c9b4c32475

In [2]:
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPTNeoForCausalLM

In [3]:
torch.manual_seed(42)

In [7]:
pretrained = "wannaphong/thaigpt-next-125m"

In [8]:
tokenizer = GPT2Tokenizer.from_pretrained(pretrained, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPTNeoForCausalLM.from_pretrained(pretrained).cuda()

In [9]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50258, 768)

In [10]:
data = [
        'Tweet: "รายละเอียดตามนี้เลยค่าา ^^" Sentiment: Neutral',
        'Tweet: "กลัวพวกมึงหาย อดกินบาบิก้อน" Sentiment: Neutral',
        'Tweet: "บริการแย่มากก เป็นหมอได้ไง😤" Sentiment: Negative',
        'Tweet: "ขับรถแย่มาก" Sentiment: Negative',
        'Tweet: "ดีนะครับ" Sentiment: Positive',
        'Tweet: "ลองแล้วรสนี้อร่อย... ชอบๆ" Sentiment: Positive',
        'Tweet: "ฉันรู้สึกโกรธ เวลามือถือแบตหมด" Sentiment: Negative',
        'Tweet: "เธอภูมิใจที่ได้ทำสิ่งดี ๆ และดีใจกับเด็ก ๆ" Sentiment: Positive',
        'Tweet: "นี่เป็นบทความหนึ่ง" Sentiment: Neutral'
]

In [11]:
max_length = max([len(tokenizer.encode(i)) for i in data])

In [12]:
max_length

89

In [13]:
class TestDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [14]:
dataset = TestDataset(data, tokenizer, max_length=max_length)

In [15]:
train_size = int(0.95 * len(dataset))

In [16]:
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [17]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=10,
                                  per_device_train_batch_size=2, per_device_eval_batch_size=2,
                                  warmup_steps=100, weight_decay=0.01, logging_dir='./logs')

In [18]:
Trainer(model=model, args=training_args, train_dataset=train_dataset,
eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])}).train()

***** Running training *****
  Num examples = 8
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 40


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=40, training_loss=0.7397451877593995, metrics={'train_runtime': 4.5809, 'train_samples_per_second': 17.464, 'train_steps_per_second': 8.732, 'total_flos': 5348516659200.0, 'train_loss': 0.7397451877593995, 'epoch': 10.0})

In [19]:
generated = tokenizer('<|startoftext|>Tweet: "มีความสุขจังเลย" Sentiment:', return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50, 
                                max_length=89, top_p=0.95, temperature=1, num_return_sequences=5)
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: <|startoftext|>Tweet: "มีความสุขจังเลย" Sentiment: Neutral
1: <|startoftext|>Tweet: "มีความสุขจังเลย" Sentiment: Negative
2: <|startoftext|>Tweet: "มีความสุขจังเลย" Sentiment: Negative
3: <|startoftext|>Tweet: "มีความสุขจังเลย" Sentiment: Negative
4: <|startoftext|>Tweet: "มีความสุขจังเลย" Sentiment: Negative
